In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [2]:
import os
from pathlib import Path

path = Path('/content/gdrive/My Drive/NLP')
os.listdir(path)

['nyt29.zip', 'Dataset']

In [3]:
extract_path = '/content/gdrive/My Drive/NLP/nyt29.zip'
import zipfile

with zipfile.ZipFile(extract_path, 'r') as zipref:
  zipref.extractall('/content/gdrive/My Drive/NLP/Dataset')

In [4]:
os.listdir('/content/gdrive/My Drive/NLP/Dataset')

['dev.pointer',
 'test.pointer',
 'train.pointer',
 '__MACOSX',
 'train_df2.gz',
 'val_df2.gz',
 'dev.sent',
 'dev.tup',
 'relations.txt',
 'test.sent',
 'test.tup',
 'train.sent',
 'train.tup',
 'Best_model.h5py']

In [5]:

import numpy as np 
import pandas as pd
from pathlib import Path
import sys
dir_path ='/content/gdrive/My Drive/NLP/Dataset'
path = Path(dir_path)
sys.path.append(dir_path)
import torch
import torch.nn as nn
import torch.nn.functional as F
from nltk.corpus import stopwords 
from collections import Counter
import string
import re
import seaborn as sns
import matplotlib.pyplot as plt
from torch.utils.data import TensorDataset, DataLoader

import random
from collections import OrderedDict
import pickle
import math
import torch.optim as optim
import datetime
import torch.autograd as autograd
from tqdm import tqdm,trange
import ast
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
file = open("{}/train.sent".format(dir_path), encoding="utf8").read()

all_sentences = []
for ind, line in enumerate(file.split('\n')):
    all_sentences.append(line)
all_sentences = all_sentences[:-1]
#to dataframe
train = pd.DataFrame({'Sentences':all_sentences})
print(train.head())

                                           Sentences
0  then terrorism struck again , this time in the...
1  a12 new york\/region b1-7 enclave for middle c...
2  before long , though , he 's continent-hopping...
3  general casey said the iraqi forces had little...
4  84 , of avon , connecticut and longboat key , ...


In [7]:
file = open("{}/dev.sent".format(dir_path), encoding="utf8").read()

all_sentences = []
for ind, line in enumerate(file.split('\n')):
    all_sentences.append(line)
all_sentences = all_sentences[:-1]
#to dataframe
val = pd.DataFrame({'Sentences':all_sentences})
print(val.head())

                                           Sentences
0  mr. scruggs -- who is arguing the case with hi...
1  officials in new delhi have even printed a glo...
2  new york city , pierson convincingly argues , ...
3  ` the wizard of oz , ' a musical by the new yo...
4  among their notable first-day selections were ...


In [10]:
file = open("{}/train.pointer".format(dir_path), encoding="utf8").read()
df_relations_list = []
for ind, line in enumerate(file.split('\n')):
    if(line.strip()==""):
      continue
    relations = line.split('| ')
    relations_list = []
    for relation in relations:
      relation_dict = dict()
      relation_word_list = relation.split(' ')
      
      relation_dict['entity1_start_index'] = relation_word_list[0]
      relation_dict['entity1_end_index'] = relation_word_list[1]
      relation_dict['entity2_start_index'] = relation_word_list[2]
      relation_dict['entity2_end_index'] = relation_word_list[3]
      relation_dict['relation'] = relation_word_list[4]

      relations_list.append(relation_dict)
    
    df_relations_list.append(relations_list)
    # print(relations_list)
    # break

train['relation_pointers'] = pd.Series(df_relations_list)

train.shape

(63306, 2)

In [11]:
file = open("{}/dev.pointer".format(dir_path), encoding="utf8").read()
df_relations_list = []
for ind, line in enumerate(file.split('\n')):
    if(line.strip()==""):
      continue
    relations = line.split('| ')
    relations_list = []
    for relation in relations:
      relation_dict = dict()
      relation_word_list = relation.split(' ')
      
      relation_dict['entity1_start_index'] = relation_word_list[0]
      relation_dict['entity1_end_index'] = relation_word_list[1]
      relation_dict['entity2_start_index'] = relation_word_list[2]
      relation_dict['entity2_end_index'] = relation_word_list[3]
      relation_dict['relation'] = relation_word_list[4]

      relations_list.append(relation_dict)
    
    df_relations_list.append(relations_list)
    # print(relations_list)
    # break

val['relation_pointers'] = pd.Series(df_relations_list)

val.shape

(7033, 2)

In [13]:
file = open("{}/train.tup".format(dir_path), encoding="utf8").read()
df_relations_list = []
for ind, line in enumerate(file.split('\n')):
    # print(line)
    if(line.strip()==""):
      continue
    relations = line.split('| ')
    relations_list = []
    for relation in relations:
      # print(relation)
      relation_dict = dict()
      relation_word_list = relation.split(' ; ')
      # print(relation_word_list)
      relation_dict['entity1_word'] = relation_word_list[0]
      relation_dict['entity2_word'] = relation_word_list[1]
      relation_dict['relation'] = relation_word_list[2].strip()

      relations_list.append(relation_dict)
    
    df_relations_list.append(relations_list)
    # print(relations_list)
    # break

train['relation_tuples'] = pd.Series(df_relations_list)

#creating relations column : 1
file = open("{}/dev.tup".format(dir_path), encoding="utf8").read()
df_relations_list = []
for ind, line in enumerate(file.split('\n')):
    # print(line)
    if(line.strip()==""):
      continue
    relations = line.split('| ')
    relations_list = []
    for relation in relations:
      # print(relation)
      relation_dict = dict()
      relation_word_list = relation.split(' ; ')
      # print(relation_word_list)
      relation_dict['entity1_word'] = relation_word_list[0]
      relation_dict['entity2_word'] = relation_word_list[1]
      relation_dict['relation'] = relation_word_list[2].strip()

      relations_list.append(relation_dict)
    
    df_relations_list.append(relations_list)
    # print(relations_list)
    # break

val['relation_tuples'] = pd.Series(df_relations_list)

In [17]:
train.head()

,Sentences,relation_pointers,relation_tuples
0,"then terrorism struck again , this time in the...","[{'entity1_start_index': '12', 'entity1_end_in...","[{'entity1_word': 'jakarta', 'entity2_word': '..."
1,a12 new york\/region b1-7 enclave for middle c...,"[{'entity1_start_index': '16', 'entity1_end_in...","[{'entity1_word': 'stuyvesant town', 'entity2_..."
2,"before long , though , he 's continent-hopping...","[{'entity1_start_index': '29', 'entity1_end_in...","[{'entity1_word': 'spain', 'entity2_word': 'pa..."
3,general casey said the iraqi forces had little...,"[{'entity1_start_index': '15', 'entity1_end_in...","[{'entity1_word': 'syria', 'entity2_word': 'eu..."
4,"84 , of avon , connecticut and longboat key , ...","[{'entity1_start_index': '10', 'entity1_end_in...","[{'entity1_word': 'florida', 'entity2_word': '..."


In [18]:
val.head()

,Sentences,relation_pointers,relation_tuples
0,mr. scruggs -- who is arguing the case with hi...,"[{'entity1_start_index': '50', 'entity1_end_in...","[{'entity1_word': 'trent lott', 'entity2_word'..."
1,officials in new delhi have even printed a glo...,"[{'entity1_start_index': '15', 'entity1_end_in...","[{'entity1_word': 'india', 'entity2_word': 'ne..."
2,"new york city , pierson convincingly argues , ...","[{'entity1_start_index': '23', 'entity1_end_in...","[{'entity1_word': 'ulster county', 'entity2_wo..."
3,"` the wizard of oz , ' a musical by the new yo...","[{'entity1_start_index': '11', 'entity1_end_in...","[{'entity1_word': 'new york', 'entity2_word': ..."
4,among their notable first-day selections were ...,"[{'entity1_start_index': '63', 'entity1_end_in...","[{'entity1_word': 'jesse barfield', 'entity2_w..."


In [19]:
train.loc[0,'relation_tuples']
val.loc[0,'relation_tuples']

[{'entity1_word': 'trent lott',
  'entity2_word': 'mississippi',
  'relation': '/people/person/place_lived'},
 {'entity1_word': 'gene taylor',
  'entity2_word': 'bay st. louis',
  'relation': '/people/person/place_lived'},
 {'entity1_word': 'trent lott',
  'entity2_word': 'pascagoula',
  'relation': '/people/person/place_lived'},
 {'entity1_word': 'gene taylor',
  'entity2_word': 'mississippi',
  'relation': '/people/person/place_lived'}]

In [20]:
def add_separators_to_sentence(sentence,first_entity_index_start,first_entity_index_end,second_entity_index_start,second_entity_index_end):
  words = sentence.split()
  words[first_entity_index_start] = '$ ' + words[first_entity_index_start]
  words[first_entity_index_end] = words[first_entity_index_end] + ' $'
  words[second_entity_index_start] = '# ' + words[second_entity_index_start]
  words[second_entity_index_end] = words[second_entity_index_end] + ' #'

  sentence_new = ' '.join(words)
  return "[CLS] " + sentence_new

# import re
# input_file = "africa african"; 
# first_entity = 'africa'
# a = re.sub(r'^' + first_entity, '$ africa $', input_file)
# print(a)

sent = 'The quick brown fox jumped over the lazy dog'
print(add_separators_to_sentence(sent,3, 3, 8,8))

[CLS] The quick brown $ fox $ jumped over the lazy # dog #


In [28]:
def preprocess_sentences_to_bert_input(row):
  sentence = row['Sentences']
  tokens = sentence.split()

  preprocessed_sentences = []
  labels = []
  entities = set()
  entities_indices = set()
  for index,relation in enumerate(row['relation_tuples']):
    # print(relation)
    entities.add(relation['entity1_word'])
    entities_indices.add((row['relation_pointers'][index]['entity1_start_index'], row['relation_pointers'][index]['entity1_end_index']))
    entities.add(relation['entity2_word'])
    entities_indices.add((row['relation_pointers'][index]['entity2_start_index'], row['relation_pointers'][index]['entity2_end_index']))
  
  entities = list(entities)
  entities_indices = list([[int(inner_item) for inner_item in item] for item in entities_indices])
  # print(entities_indices)
  for i in range(len(entities)):
    for j in range(i+1,len(entities)):
      relations = []
      for index,relation in enumerate(row['relation_tuples']):
        if(entities[i]==relation['entity1_word'] and entities[j]==relation['entity2_word']):
          relations.append(relation['relation'])
      preprocessed_sentence  = add_separators_to_sentence(sentence,entities_indices[i][0],entities_indices[i][1],entities_indices[j][0],entities_indices[j][1])
      if(len(relations)==0):
        relations.append("None")
      preprocessed_sentences.append(preprocessed_sentence)
      labels.append(relations)

      relations = []
      for relation in row['relation_tuples']:
        if(entities[j]==relation['entity1_word'] and entities[i]==relation['entity2_word']):
          relations.append(relation['relation'])
      preprocessed_sentence =  add_separators_to_sentence(sentence,entities_indices[j][0],entities_indices[j][1],entities_indices[i][0],entities_indices[i][1])
      if(len(relations)==0):
        relations.append("None")
      preprocessed_sentences.append(preprocessed_sentence)
      labels.append(relations)

  d = dict()
  d['preprocessed_sentences'] = preprocessed_sentences
  d['labels'] = labels
  return d

In [29]:
train['preprocessed_sentences_labels'] = train.apply(preprocess_sentences_to_bert_input,axis=1)

In [30]:
val['preprocessed_sentences_labels'] = val.apply(preprocess_sentences_to_bert_input,axis=1)

In [31]:
train.loc[0,'preprocessed_sentences_labels']

{'labels': [['/location/country/capital',
   '/location/country/administrative_divisions'],
  ['/location/administrative_division/country']],
 'preprocessed_sentences': ['[CLS] then terrorism struck again , this time in the # indonesia # capital of $ jakarta $ .',
  '[CLS] then terrorism struck again , this time in the $ indonesia $ capital of # jakarta # .']}

In [32]:
val.loc[0,'preprocessed_sentences_labels']

{'labels': [['None'],
  ['None'],
  ['/people/person/place_lived'],
  ['None'],
  ['None'],
  ['None'],
  ['/people/person/place_lived'],
  ['None'],
  ['None'],
  ['None'],
  ['/people/person/place_lived'],
  ['None'],
  ['/people/person/place_lived'],
  ['None'],
  ['None'],
  ['None'],
  ['None'],
  ['None'],
  ['None'],
  ['None']],
 'preprocessed_sentences': ['[CLS] mr. scruggs -- who is arguing the case with his son zach ; a colleague from northern $ mississippi $ , don barrett ; and john jones from jackson , miss. -- said he had already filed suit on behalf of about 2,000 clients seeking redress from their insurers , including senator # trent lott # , who is both a neighbor of mr. scruggs in pascagoula and his brother-in-law ; and representative gene taylor of bay st. louis .',
  '[CLS] mr. scruggs -- who is arguing the case with his son zach ; a colleague from northern # mississippi # , don barrett ; and john jones from jackson , miss. -- said he had already filed suit on behal

In [33]:
train.head()

,Sentences,relation_pointers,relation_tuples,preprocessed_sentences_labels
0,"then terrorism struck again , this time in the...","[{'entity1_start_index': '12', 'entity1_end_in...","[{'entity1_word': 'jakarta', 'entity2_word': '...",{'preprocessed_sentences': ['[CLS] then terror...
1,a12 new york\/region b1-7 enclave for middle c...,"[{'entity1_start_index': '16', 'entity1_end_in...","[{'entity1_word': 'stuyvesant town', 'entity2_...",{'preprocessed_sentences': ['[CLS] a12 new yor...
2,"before long , though , he 's continent-hopping...","[{'entity1_start_index': '29', 'entity1_end_in...","[{'entity1_word': 'spain', 'entity2_word': 'pa...",{'preprocessed_sentences': ['[CLS] before long...
3,general casey said the iraqi forces had little...,"[{'entity1_start_index': '15', 'entity1_end_in...","[{'entity1_word': 'syria', 'entity2_word': 'eu...",{'preprocessed_sentences': ['[CLS] general cas...
4,"84 , of avon , connecticut and longboat key , ...","[{'entity1_start_index': '10', 'entity1_end_in...","[{'entity1_word': 'florida', 'entity2_word': '...","{'preprocessed_sentences': ['[CLS] 84 , of avo..."


In [34]:
val.head()

,Sentences,relation_pointers,relation_tuples,preprocessed_sentences_labels
0,mr. scruggs -- who is arguing the case with hi...,"[{'entity1_start_index': '50', 'entity1_end_in...","[{'entity1_word': 'trent lott', 'entity2_word'...",{'preprocessed_sentences': ['[CLS] mr. scruggs...
1,officials in new delhi have even printed a glo...,"[{'entity1_start_index': '15', 'entity1_end_in...","[{'entity1_word': 'india', 'entity2_word': 'ne...",{'preprocessed_sentences': ['[CLS] officials i...
2,"new york city , pierson convincingly argues , ...","[{'entity1_start_index': '23', 'entity1_end_in...","[{'entity1_word': 'ulster county', 'entity2_wo...",{'preprocessed_sentences': ['[CLS] new york ci...
3,"` the wizard of oz , ' a musical by the new yo...","[{'entity1_start_index': '11', 'entity1_end_in...","[{'entity1_word': 'new york', 'entity2_word': ...",{'preprocessed_sentences': ['[CLS] ` the wizar...
4,among their notable first-day selections were ...,"[{'entity1_start_index': '63', 'entity1_end_in...","[{'entity1_word': 'jesse barfield', 'entity2_w...",{'preprocessed_sentences': ['[CLS] among their...


In [35]:
train_2 = pd.DataFrame(columns=['sentence_index','sentence','preprocessed_sentence','labels'])

In [36]:
for index,row in tqdm(train.iterrows()):
  temp = pd.DataFrame(columns=['sentence_index','sentence','preprocessed_sentence','labels'])
  sentence = row['Sentences']
  preprocessed_sentences = row['preprocessed_sentences_labels']['preprocessed_sentences']
  labels = row['preprocessed_sentences_labels']['labels']
  for i in range(len(labels)):
    data = pd.Series({'sentence_index': index,'sentence':sentence,"preprocessed_sentence": preprocessed_sentences[i], "labels": labels[i]})
    temp = temp.append(data,ignore_index=True)
  
  train_2 = train_2.append(temp,ignore_index = True)

63306it [19:29, 54.14it/s]


In [37]:
val_2 = pd.DataFrame(columns=['sentence_index','sentence','preprocessed_sentence','labels'])

In [38]:
for index,row in tqdm(val.iterrows()):
  temp = pd.DataFrame(columns=['sentence_index','sentence','preprocessed_sentence','labels'])
  sentence = row['Sentences']
  preprocessed_sentences = row['preprocessed_sentences_labels']['preprocessed_sentences']
  labels = row['preprocessed_sentences_labels']['labels']
  for i in range(len(labels)):
    data = pd.Series({'sentence_index': index,'sentence':sentence,"preprocessed_sentence": preprocessed_sentences[i], "labels": labels[i]})
    temp = temp.append(data,ignore_index=True)
  
  val_2 = val_2.append(temp,ignore_index = True)

7033it [01:11, 98.52it/s]


In [39]:
train.shape , val.shape

((63306, 4), (7033, 4))

In [40]:
train_2.to_csv('{}/train.gz'.format(dir_path), compression='gzip')
val_2.to_csv('{}/val.gz'.format(dir_path), compression='gzip')

In [43]:
os.listdir(dir_path)

['dev.pointer',
 'test.pointer',
 'train.pointer',
 '__MACOSX',
 'train_df2.gz',
 'val_df2.gz',
 'dev.sent',
 'dev.tup',
 'relations.txt',
 'test.sent',
 'test.tup',
 'train.sent',
 'train.tup',
 'Best_model.h5py',
 'train.gz',
 'val.gz']